기존 데이터와 시뮬레이션을 통해 생성된 데이터를 통해 냉해로 인한 과수피해의 평균 발생 횟수 $\lambda$를 추정하고자 한다.

# 기본 세팅

In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (9,143 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120875 files and dire

In [2]:
! pip install catboost
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.4 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import numpy as np
import random
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)


from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from catboost import CatBoostRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import RobustScaler
from scipy.signal import savgol_filter
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler  # 표준화 패키지 라이브러리
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.preprocessing import MinMaxScaler

import warnings

import random

# 모든 경고 메시지 끄기
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumBarunGothic' # 나눔바른고딕 적용하기


# 데이터 불러오기

In [5]:
# 기상 데이터

actual = pd.read_csv('/content/drive/MyDrive/기상보간iter.csv',encoding = 'UTF-8')
simulated = pd.read_csv('/content/drive/MyDrive/기상 시뮬레이션 데이터.csv',encoding = 'UTF-8')
actual_outlier = pd.read_csv('/content/drive/MyDrive/최저기온이상치_최종최종이거.csv',encoding = 'UTF-8')

print(actual.columns)
print(simulated.columns)

simulated.drop('Unnamed: 0',axis = 1,inplace = True)
simulated.rename(columns = {'month':'월'},inplace = True)

Index(['일 최심적설(cm)', '일시', '합계 3시간 신적설(cm)', '안개 계속시간(hr)', '일강수량(mm)',
       '일 최심신적설(cm)', '지점', '지점명', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)',
       '최대 순간 풍속(m/s)', '최대 순간 풍속 풍향(16방위)', '최대 풍속(m/s)', '최대 풍속 풍향(16방위)',
       '평균 풍속(m/s)', '풍정합(100m)', '최다풍향(16방위)', '평균 이슬점온도(°C)', '최소 상대습도(%)',
       '평균 상대습도(%)', '평균 증기압(hPa)', '평균 현지기압(hPa)', '최고 해면기압(hPa)',
       '최저 해면기압(hPa)', '평균 해면기압(hPa)', '가조시간(hr)', '합계 일조시간(hr)',
       '평균 전운량(1/10)', '평균 지면온도(°C)', '최저 초상온도(°C)', '해양'],
      dtype='object')
Index(['Unnamed: 0', '일 최심적설(cm)', '합계 3시간 신적설(cm)', '일강수량(mm)', '일 최심신적설(cm)',
       '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '최대 순간 풍속(m/s)', '최대 풍속(m/s)',
       '평균 풍속(m/s)', '평균 이슬점온도(°C)', '최소 상대습도(%)', '평균 상대습도(%)', '평균 증기압(hPa)',
       '평균 현지기압(hPa)', '합계 일조시간(hr)', '평균 전운량(1/10)', '평균 지면온도(°C)',
       '최저 초상온도(°C)', '지점명', 'month', 'counter'],
      dtype='object')


# 이상치 탐지

In [6]:
# SARIMAX: 각 지점별, counter별로 한 경우
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tqdm import tqdm

def detect_anomalities_arima(data_iter_cold):
    result_df = pd.DataFrame(columns=['지점명', '최저기온(°C)', 'anomality','counter'])
    unique_locations = data_iter_cold['지점명'].unique()
    unique_counters = data_iter_cold['counter'].unique()

    for location in tqdm(unique_locations, desc="Processing Locations"):
        for counter in unique_counters:
            location_data = data_iter_cold[(data_iter_cold['지점명'] == location) & (data_iter_cold['counter'] == counter)][['최저기온(°C)']]
            #location_data.set_index('월', inplace=True)

            # SARIMAX 모델 학습
            model = SARIMAX(location_data['최저기온(°C)'], order=(1, 1, 0), seasonal_order=(1, 0, 0, 3))
            model_fit = model.fit(disp=0)

            # 예측 및 잔차 계산
            predictions = model_fit.predict(typ='levels')
            residuals = location_data['최저기온(°C)'] - predictions

            location_data['anomality'] = np.where(np.abs(residuals) > np.abs(residuals).quantile(0.9), -1, 1) # residual이 클수록 실제 데이터와 멀기 때문
            location_data['지점명'] = location
            location_data['counter'] = counter
            result_df = result_df.append(location_data[['지점명', '최저기온(°C)', 'anomality', 'counter']])

    return result_df


# 결과 저장
result_df_arima = detect_anomalities_arima(simulated)

Processing Locations:   0%|          | 0/86 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and w

In [7]:
# Isolation Forest: 각 지역별, counter별 한 경우
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import IsolationForest


def remove_trend_and_seasonality(data):
    decomposition = seasonal_decompose(data, model='additive', period=3)
    residual = data - decomposition.trend - decomposition.seasonal
    residual = residual.fillna(method='ffill').fillna(method='bfill')
    return residual

def detect_anomalities_isolation_forest(data_iter_cold):
    result_df = pd.DataFrame(columns=['지점명', '최저기온(°C)', 'anomality','counter'])
    unique_locations = data_iter_cold['지점명'].unique()
    unique_counters = data_iter_cold['counter'].unique()

    for location in tqdm(unique_locations, desc="Processing Locations"):
        for counter in unique_counters:
          location_data = data_iter_cold[(data_iter_cold['지점명'] == location) & (data_iter_cold['counter'] == counter)][['최저기온(°C)']]

          residual = remove_trend_and_seasonality(location_data['최저기온(°C)'])
          residual = residual.values.reshape(-1, 1)

          isolation_forest = IsolationForest(contamination=0.15, n_estimators = 100, max_samples= 'auto', n_jobs = -1, random_state = 1)

          # Fit the model and predict anomalies (1 indicates anomaly, -1 indicates normal)
          location_data['anomality'] = isolation_forest.fit_predict(residual)
          location_data['지점명'] = location

          result_df = result_df.append(location_data)
    return result_df

# 결과 저장
result_df_isolation_forest = detect_anomalities_isolation_forest(simulated)

# 각 지역별로 anomality 갯수 확인
(result_df_isolation_forest[result_df_isolation_forest['anomality'] == 1].groupby('지점명')['anomality'].count() /
 result_df_isolation_forest.groupby('지점명')['anomality'].count())

Processing Locations: 100%|██████████| 86/86 [02:57<00:00,  2.06s/it]


지점명
강릉     0.843750
강화     0.843750
거제     0.843750
거창     0.843750
고산     0.843750
         ...   
합천     0.845486
해남     0.843750
홍성     0.843750
홍천     0.843750
흑산도    0.843750
Name: anomality, Length: 86, dtype: float64

In [8]:
#result_df_isolation_forest.drop_duplicates(inplace = True)
simulated['sarimax_anomality'] =result_df_arima['anomality']
simulated['isoforest_anomality'] =result_df_isolation_forest['anomality']

# 데이터 전처리

In [15]:
# counter에 따라 데이터 분리

by_counter = {
    '0': simulated[simulated['counter']==1].drop('counter',axis =1),

    '0.2': simulated[simulated['counter']==2].drop('counter',axis =1),

    '0.4': simulated[simulated['counter']==3].drop('counter',axis =1),

    '0.6': simulated[simulated['counter']==4].drop('counter',axis =1),

    '0.8': simulated[simulated['counter']==5].drop('counter',axis =1),

    '1': simulated[simulated['counter']==6].drop('counter',axis =1)}

In [16]:
# '일' 열 추가

for i in by_counter.keys():
  row_count = by_counter[i].shape[0]
  by_counter[i]['일'] = list(range(1, 33)) * (row_count // 32) + list(range(1, row_count % 32 + 1))

# 삭제해야할 행 조건

# 조건1: '일' 열이 32인 행 삭제
for key, df in by_counter.items():
    condition1 = df['일'] == 32
    by_counter[key] = df[~condition1]

# 조건2: 'month'가 2이면서 '일'이 29, 30, 31 중 하나인 행 삭제
for key, df in by_counter.items():
    condition2 = (df['월'] == 2) & (df['일'].isin([29, 30, 31]))
    by_counter[key] = df[~condition2]

# 조건3: 'month'가 4이고 '일'이 31인 행 삭제
for key, df in by_counter.items():
    condition3 = (df['월'] == 4) & (df['일'] == 31)
    by_counter[key] = df[~condition3]


In [17]:
# 연도 추가

for i in by_counter.keys():
  row_count = by_counter[i].shape[0]
  by_counter[i]['연'] = np.repeat(2024,row_count)

In [18]:
coord= pd.read_csv('/content/drive/MyDrive/지점명_좌표.csv',encoding='CP949')
def wider(data):

    # 'data' 데이터프레임과 'coord' 데이터프레임을 '지점명'을 기준으로 합치고 누락된 데이터 제거
    data = pd.merge(data, coord, on='지점명', how='left')
    data = data.dropna()

    # 월이 2 또는 3인 데이터만 선택
    data = data[data['월'].isin([2, 3])]

    # 일교차 열 추가
    data['일교차'] = data['최고기온(°C)'] - data['최저기온(°C)']

    # 'stata' 데이터프레임 생성
    stata = data.groupby(['연','월', '지점명', '위도', '경도', '고도']).agg({
        '최고기온(°C)': ['min', 'max', 'mean'],
        '평균기온(°C)': ['min', 'max', 'mean'],
        '최저기온(°C)': ['min', 'max', 'mean'],
        '일강수량(mm)': 'sum',
        '합계 일조시간(hr)': 'sum'
    }).reset_index()

    # 열 이름 업데이트
    stata.columns = [f'{col[0]}_{col[1]}' if col[1] else col[0] for col in stata.columns]

    # '지점명'을 '지점'으로 변경
    stata.rename(columns={'지점명': '지점'}, inplace=True)

    # '연'과 '지점'을 기준으로 pivot_table을 사용하여 '월' 정보를 와이드 포맷으로 변경
    values = ['최고기온(°C)_min', '최고기온(°C)_max',
              '최고기온(°C)_mean', '평균기온(°C)_min', '평균기온(°C)_max', '평균기온(°C)_mean',
              '최저기온(°C)_min', '최저기온(°C)_max', '최저기온(°C)_mean', '일강수량(mm)_sum',
              '합계 일조시간(hr)_sum']

    wide_format = stata.pivot_table(index=['연','지점', '위도', '경도', '고도'], columns='월', values=values)
    wide_format.reset_index(inplace=True)

    # 열 이름을 원하는 형식으로 변경
    wide_format.columns = [f'{str(col[1])}_{col[0]}' if col[1] else col[0] for col in wide_format.columns]

    wide_format.reset_index(inplace=True)
    wide_format.dropna(inplace=True)

    return wide_format

In [19]:
# 개화시기 예측을 위한 데이터

blossom = {}
for key, df in by_counter.items():
    blossom[key] = wider(df)

In [20]:
blossom.keys()

dict_keys(['0', '0.2', '0.4', '0.6', '0.8', '1'])

# 개화일 예측

## 모델 세팅

In [21]:
peach = pd.read_csv('/content/drive/MyDrive/복숭아 개화 데이터.csv',encoding='cp949')
pear = pd.read_csv('/content/drive/MyDrive/배 개화 데이터.csv',encoding='cp949')

x_col=[ '위도', '경도', '고도', '2_일강수량(mm)_sum', '3_일강수량(mm)_sum',
       '2_최고기온(°C)_max', '3_최고기온(°C)_max', '2_최고기온(°C)_mean',
       '3_최고기온(°C)_mean', '2_최고기온(°C)_min', '3_최고기온(°C)_min', '2_최저기온(°C)_max',
       '3_최저기온(°C)_max', '2_최저기온(°C)_mean', '3_최저기온(°C)_mean',
       '2_최저기온(°C)_min', '3_최저기온(°C)_min', '2_평균기온(°C)_max', '3_평균기온(°C)_max',
       '2_평균기온(°C)_mean', '3_평균기온(°C)_mean', '2_평균기온(°C)_min',
       '3_평균기온(°C)_min', '2_합계 일조시간(hr)_sum', '3_합계 일조시간(hr)_sum']

y_col='365'

In [22]:
def cv(model,x_data,y_data,metric='both'):
  X = x_data
  y = y_data

  kf = KFold(n_splits=5, shuffle=True, random_state=42)

  mae_scores = []
  rmse_scores = []

  for train_index, test_index in kf.split(X):
      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]

      model.fit(X_train, y_train)

      y_pred = model.predict(X_test)

      mae = mean_absolute_error(y_test, y_pred)
      rmse = np.sqrt(mean_squared_error(y_test, y_pred))

      mae_scores.append(mae)
      rmse_scores.append(rmse)

  mean_mae = np.mean(mae_scores)
  mean_rmse = np.mean(rmse_scores)

  score = {'mae': mean_mae,'rmse':mean_rmse }

  if metric == 'both':
    result = score
  else:
    result = score[metric]

  return result

In [23]:
# print(cv(x_data=peach[x_col],y_data=peach[y_col],model=lgb.LGBMRegressor(random_state=6))) # 복숭아

# {'mae': 3.594324670100118, 'rmse': 5.012884818236105}

# print(cv(x_data=pear[x_col],y_data=pear[y_col],model=lgb.LGBMRegressor(random_state=6))) # 배

# {'mae': 3.5148556434264995, 'rmse': 4.818330202088425}

In [24]:
def lgbm_objective(trial):



  lgbm_param = { 'objective': 'regression', # 회귀
          'verbose': -1,
          'metric': 'rmse',
          'max_depth': trial.suggest_int('max_depth',3, 15),
          'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
          'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
          'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
          'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
      }

  lgbm = lgb.LGBMRegressor(**lgbm_param,random_state=6)

  score = cv(x_data=data[x_col],y_data=data[y_col],model=lgbm,metric='mae')

  return score

## 복숭아 & 배 모델

In [25]:
"""
data = peach.copy()
peach_lgbm_study = optuna.create_study(direction='minimize', sampler=TPESampler())
peach_lgbm_study.optimize(lgbm_objective,n_trials=100,timeout = 1800)
trial = peach_lgbm_study.best_trial
peach_params = trial.params
print('Best Trial: score {},\nparams {}'.format(trial.value, peach_params))

#Best Trial: score 3.5422616806304874,
#params {'max_depth': 10, 'learning_rate': 0.00997460419066651, 'n_estimators': 2977, 'min_child_samples': 6, 'subsample': 0.8770895855246077}

peach_lgbm = lgb.LGBMRegressor(**peach_params,random_state=6)

"""

"\ndata = peach.copy()\npeach_lgbm_study = optuna.create_study(direction='minimize', sampler=TPESampler())\npeach_lgbm_study.optimize(lgbm_objective,n_trials=100,timeout = 1800)\ntrial = peach_lgbm_study.best_trial\npeach_params = trial.params\nprint('Best Trial: score {},\nparams {}'.format(trial.value, peach_params))\n\n#Best Trial: score 3.5422616806304874,\n#params {'max_depth': 10, 'learning_rate': 0.00997460419066651, 'n_estimators': 2977, 'min_child_samples': 6, 'subsample': 0.8770895855246077}\n\npeach_lgbm = lgb.LGBMRegressor(**peach_params,random_state=6)\n\n"

In [26]:
"""
data = pear.copy()
pear_lgbm_study = optuna.create_study(direction='minimize', sampler=TPESampler())
pear_lgbm_study.optimize(lgbm_objective,n_trials=100,timeout = 1800)
trial = pear_lgbm_study.best_trial
pear_params = trial.params
print('Best Trial: score {},\nparams {}'.format(trial.value, pear_params))

#Best Trial: score 3.4510470942184526,
#params {'max_depth': 12, 'learning_rate': 0.005420837898127981, 'n_estimators': 2396, 'min_child_samples': 37, 'subsample': 0.5146303099739187}

pear_lgbm = lgb.LGBMRegressor(**pear_params,random_state=6)
"""

"\ndata = pear.copy()\npear_lgbm_study = optuna.create_study(direction='minimize', sampler=TPESampler())\npear_lgbm_study.optimize(lgbm_objective,n_trials=100,timeout = 1800)\ntrial = pear_lgbm_study.best_trial\npear_params = trial.params\nprint('Best Trial: score {},\nparams {}'.format(trial.value, pear_params))\n\n#Best Trial: score 3.4510470942184526,\n#params {'max_depth': 12, 'learning_rate': 0.005420837898127981, 'n_estimators': 2396, 'min_child_samples': 37, 'subsample': 0.5146303099739187}\n\npear_lgbm = lgb.LGBMRegressor(**pear_params,random_state=6)\n"

In [27]:
peach_params = {'max_depth': 10,
                'learning_rate': 0.00997460419066651,
                'n_estimators': 2977,
                'min_child_samples': 6,
                'subsample': 0.8770895855246077}

pear_params = {'max_depth': 12,
               'learning_rate': 0.005420837898127981,
               'n_estimators': 2396,
               'min_child_samples': 37,
               'subsample': 0.5146303099739187}

peach_lgbm = lgb.LGBMRegressor(**peach_params,random_state=6)
pear_lgbm = lgb.LGBMRegressor(**pear_params,random_state=6)

In [28]:
peach_lgbm.fit(peach[x_col],peach[y_col])
pear_lgbm.fit(pear[x_col],pear[y_col])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000985 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4543
[LightGBM] [Info] Number of data points in the train set: 1885, number of used features: 25
[LightGBM] [Info] Start training from score 98.861538
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

LGBMRegressor(learning_rate=0.005420837898127981, max_depth=12,
              min_child_samples=37, n_estimators=2396, random_state=6,
              subsample=0.5146303099739187)

In [29]:
for key, df in blossom.items():

    # 각 모델을 사용하여 예측 수행
    peach_predictions = peach_lgbm.predict(df[x_col])
    pear_predictions = pear_lgbm.predict(df[x_col])

    # 예측 결과를 데이터프레임에 추가
    df['peach_predictions'] = peach_predictions
    df['pear_predictions'] = pear_predictions

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^m

In [30]:
blossom['0']

,index,연,지점,위도,경도,고도,2_일강수량(mm)_sum,3_일강수량(mm)_sum,2_최고기온(°C)_max,3_최고기온(°C)_max,...,2_평균기온(°C)_max,3_평균기온(°C)_max,2_평균기온(°C)_mean,3_평균기온(°C)_mean,2_평균기온(°C)_min,3_평균기온(°C)_min,2_합계 일조시간(hr)_sum,3_합계 일조시간(hr)_sum,peach_predictions,pear_predictions
0,0,2024,강릉,37.762825,128.935700,26.290000,52.123880,57.043902,8.787593,17.908805,...,4.768432,10.924988,0.273193,5.307069,-3.776416,-2.884122,139.558634,190.333301,100.465699,102.706426
1,1,2024,강화,37.707400,126.446300,47.425000,3.784963,37.117584,12.162223,15.319184,...,5.916261,8.955938,-2.057812,3.394620,-6.860933,-3.445832,162.929473,228.270669,109.622991,111.965658
2,2,2024,거제,34.888200,128.604600,44.830000,72.843761,113.924828,13.030259,16.174874,...,9.404040,10.350010,3.603999,6.987175,-1.628259,1.156183,164.127554,225.574685,93.549401,98.084481
3,3,2024,거창,35.669300,127.910450,224.925000,26.694670,32.895680,11.322040,18.324010,...,5.119874,8.812341,-0.758096,3.880108,-6.213359,-2.675797,184.271566,219.434229,105.516957,105.715557
4,4,2024,고산,33.963550,126.424900,68.242500,-6.108318,84.215383,13.924388,15.941215,...,10.698869,11.990077,5.299680,8.162702,-0.471959,1.990406,126.134347,163.785350,92.936830,94.677167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,81,2024,합천,35.561200,128.161233,33.523333,22.323292,70.736567,13.514623,16.944310,...,6.498812,9.936642,0.275084,4.893127,-4.247915,-1.409703,171.190455,199.767294,102.686677,101.618929
82,82,2024,해남,34.561150,126.441800,28.180000,-1.500385,70.105781,12.173677,16.372893,...,8.197369,9.592423,1.354665,5.186145,-2.935452,-2.954216,147.225616,217.483924,101.488090,97.726993
83,83,2024,홍성,36.610067,126.654033,44.670000,39.326067,15.148627,15.641685,19.516100,...,7.716420,12.983120,0.580127,6.614384,-5.348076,0.131938,166.254205,240.987466,101.211529,101.636651
84,84,2024,홍천,37.683600,127.880400,140.200000,-15.954875,51.110465,12.880409,16.396261,...,4.843132,7.256422,-3.834143,1.869600,-10.850775,-5.688012,198.525132,224.493447,115.655061,114.798913


# 냉해 발생 횟수 카운트

- 이상치 탐지 결과와 개화일 예측 결과 합치기
  - 기존 실제 데이터
  - 생성된 데이터
- 이상치 탐지된 날짜들 365 단위로 변환
- 개화일 < 이상치 365 -> 냉해 발생

- n = 데이터셋 행의 수
- p = 냉해의 발생 확률
  - np = λ : n이 크고 p가 작을때 이항분포는 포아송 분포로 근사됨

## 생성된 데이터의 case

### 데이터 합치기

In [31]:
# 우선 이상치 데이터와 개화일 예측 데이터 합치기

result = {}

for key in by_counter.keys():
  day365 = blossom[key]
  day365= day365[['연','지점','peach_predictions','pear_predictions']]
  day365.rename(columns = {'지점':'지점명'},inplace = True)
  result[key]=pd.merge(by_counter[key],day365,on= ['연','지점명'], how = 'left')

In [32]:
from datetime import datetime

def date_365(year, month , day):

    # 1월 1일부터의 일차 계산
    date_str = f'{year}-{month}-{day}'
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")

    start_date = datetime(date_obj.year, 1, 1)
    day_number = (date_obj - start_date).days + 1

    return day_number

for df in result.values():
  df['날짜']=df.apply(lambda row: date_365(row['연'], row['월'], row['일']), axis=1)

In [33]:
def count_anomalous_rows(data, anomality, prediction):
    # '지점명'으로 그룹화하고 행 전체의 개수를 세는 함수
    def count_total_rows(group):
        return len(group)

    # 'anomality'가 -1이며 '날짜'가 'prediction'보다 큰 행들의 개수를 세는 함수
    def count_anomalous_and_date_rows(group):
        return len(group[(group[anomality] == -1) & (group['날짜'] > group[prediction])])

    # '지점명'으로 그룹화하고 각 그룹에 대해 함수 적용
    total_counts = data.groupby('지점명').apply(count_total_rows).reset_index(name='total_count')
    anomalous_counts = data.groupby('지점명').apply(count_anomalous_and_date_rows).reset_index(name='anomalous_count')

    # 결과를 병합하여 반환
    result = pd.merge(total_counts, anomalous_counts, on='지점명', how='left')
    return result

In [34]:
result['0']['isoforest_anomality'].value_counts()

 1    6457
-1    1197
Name: isoforest_anomality, dtype: int64

### anomality 계산

In [35]:
peach_anomality = {}
for key,df in result.items():
  smx = count_anomalous_rows(df,anomality = 'sarimax_anomality',prediction='peach_predictions')
  smx.rename(columns = {'anomalous_count':'sarimax'},inplace =True)

  iso = count_anomalous_rows(df,anomality = 'isoforest_anomality',prediction='peach_predictions')
  iso.rename(columns = {'anomalous_count':'isoforest'},inplace =True)

  peach_anomality[key]= pd.merge(smx,iso,on = ['지점명','total_count'],how='left')

In [36]:
pear_anomality = {}
for key,df in result.items():
  smx = count_anomalous_rows(df,anomality = 'sarimax_anomality',prediction='pear_predictions')
  smx.rename(columns = {'anomalous_count':'sarimax'},inplace =True)

  iso = count_anomalous_rows(df,anomality = 'isoforest_anomality',prediction='pear_predictions')
  iso.rename(columns = {'anomalous_count':'isoforest'},inplace =True)

  pear_anomality[key]= pd.merge(smx,iso,on = ['지점명','total_count'],how='left')

In [37]:
peach_agg = peach_anomality['0']
pear_agg = pear_anomality['0']

counter = ['0.2','0.4','0.6','0.8','1']

for i in counter:
  df= peach_anomality[i]
  df= df.rename(columns={'sarimax': f'{i}_sarimax', 'isoforest': f'{i}_isoforest'})

  df2= pear_anomality[i]
  df2= df.rename(columns={'sarimax': f'{i}_sarimax', 'isoforest': f'{i}_isoforest'})

  peach_agg = pd.concat([peach_agg,df[[f'{i}_sarimax',f'{i}_isoforest']]],axis=1)
  pear_agg = pd.concat([pear_agg,df2[[f'{i}_sarimax',f'{i}_isoforest']]],axis=1)


In [38]:
peach_agg.sort_values(by=['sarimax', 'isoforest'], ascending=False)

,지점명,total_count,sarimax,isoforest,0.2_sarimax,0.2_isoforest,0.4_sarimax,0.4_isoforest,0.6_sarimax,0.6_isoforest,0.8_sarimax,0.8_isoforest,1_sarimax,1_isoforest
39,성산,89,7,8,4,8,4,6,5,6,4,6,5,5
40,성산포,89,5,8,4,6,3,10,4,7,5,8,4,8
7,고흥,89,4,5,3,4,2,1,5,4,2,6,6,8
23,밀양,89,4,5,1,2,2,3,2,2,0,3,3,0
54,울진,89,4,5,4,4,2,2,3,6,1,4,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,장수,89,0,0,1,3,1,0,1,3,0,0,1,1
72,철원,89,0,0,1,0,2,4,2,3,0,0,0,0
74,추풍령,89,0,0,0,0,2,1,1,0,1,2,3,5
77,태백,89,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
pear_agg.sort_values(by=['sarimax', 'isoforest'], ascending=False)

,지점명,total_count,sarimax,isoforest,0.2_sarimax,0.2_isoforest,0.4_sarimax,0.4_isoforest,0.6_sarimax,0.6_isoforest,0.8_sarimax,0.8_isoforest,1_sarimax,1_isoforest
39,성산,89,6,6,4,8,4,6,5,6,4,6,5,5
40,성산포,89,4,7,4,6,3,10,4,7,5,8,4,8
23,밀양,89,4,4,1,2,2,3,2,2,0,3,3,0
64,정읍,89,4,4,4,2,2,4,2,3,0,3,2,1
81,합천,89,4,2,0,1,1,5,3,2,2,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,철원,89,0,0,1,0,2,4,2,3,0,0,0,0
73,청주,89,0,0,0,2,2,3,0,1,0,2,3,4
74,추풍령,89,0,0,0,0,2,1,1,0,1,2,3,5
77,태백,89,0,0,0,0,0,0,0,0,0,0,0,0


## 실제 데이터의 경우

In [40]:
# actual 데이터와 최저기온이상치_최종최종을 합친다.

actual_outlier.rename(columns = {'SARIMAX':'sarimax_anomality',
                                 'isolation':'isoforest_anomality',
                                 'Year':'연'},inplace = True)

peach365 =peach[['연','지점','365']]
peach365.rename(columns = {'지점':'지점명','365':'peach365'},inplace = True)

pear365 = pear[['연','지점','365']]
pear365.rename(columns = {'지점':'지점명','365':'pear365'},inplace = True)


actual_result = pd.merge(actual_outlier,peach365,on= ['연','지점명'], how = 'right')
actual_result = pd.merge(actual_result,pear365,on= ['연','지점명'], how = 'left')

In [41]:
def date_to_day_number(date_str):
    # 날짜 문자열을 datetime 객체로 변환
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")

    # 1월 1일부터의 일차 계산
    start_date = datetime(date_obj.year, 1, 1)
    day_number = (date_obj - start_date).days + 1

    return day_number

actual_result['날짜']=actual_result['일시'].apply(date_to_day_number)

In [42]:
actual_result

,일 최심적설(cm),일시,합계 3시간 신적설(cm),안개 계속시간(hr),일강수량(mm),일 최심신적설(cm),지점,지점명,평균기온(°C),최저기온(°C),...,평균 전운량(1/10),평균 지면온도(°C),최저 초상온도(°C),해양,연,sarimax_anomality,isoforest_anomality,peach365,pear365,날짜
0,0.0,1989-02-01,0.0,0.0,0.0,0.0,105.0,강릉,1.1,-3.0,...,2.500000,0.6,-8.4,1,1989,1,1,95,107.0,32
1,0.0,1989-02-02,0.0,0.0,0.0,0.0,105.0,강릉,-2.1,-6.2,...,0.000000,-1.0,-10.2,1,1989,1,1,95,107.0,33
2,0.0,1989-02-03,0.0,0.0,0.0,0.0,105.0,강릉,-1.5,-6.1,...,0.000000,-0.1,-9.4,1,1989,1,1,95,107.0,34
3,0.0,1989-02-04,0.0,0.0,0.0,0.0,105.0,강릉,0.8,-4.1,...,0.000000,2.0,-6.6,1,1989,1,1,95,107.0,35
4,0.0,1989-02-05,0.0,0.0,0.0,0.0,105.0,강릉,2.5,-0.4,...,3.800000,2.9,-4.6,1,1989,1,1,95,107.0,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174140,0.0,2014-04-26,0.0,0.0,0.0,0.0,127.0,충주,16.8,8.9,...,7.045615,17.7,8.6,0,2014,1,1,100,97.0,116
174141,0.0,2014-04-27,0.0,0.0,6.2,0.0,127.0,충주,14.9,13.0,...,10.309957,15.8,12.7,0,2014,1,1,100,97.0,117
174142,0.0,2014-04-28,0.0,0.0,14.8,0.0,127.0,충주,13.2,12.1,...,10.492841,13.3,11.7,0,2014,1,1,100,97.0,118
174143,0.0,2014-04-29,0.0,0.0,1.9,0.0,127.0,충주,15.1,10.6,...,7.009998,14.5,10.7,0,2014,1,1,100,97.0,119


In [57]:
def count_anomalous_rows2(data, anomality, prediction):
    # '연', '지점명'으로 그룹화하고 행 전체의 개수를 세는 함수
    def count_total_rows(group):
        return len(group)

    # 'anomality'가 -1이며 '날짜'가 'prediction'보다 큰 행들의 개수를 세는 함수
    def count_anomalous_and_date_rows(group):
        return len(group[(group[anomality] == -1) & (group['날짜'] > group[prediction])])

    # '연', '지점명'으로 그룹화하고 각 그룹에 대해 함수 적용
    total_counts = data.groupby(['연', '지점명']).apply(count_total_rows).reset_index(name='total_count')
    anomalous_counts = data.groupby(['연', '지점명']).apply(count_anomalous_and_date_rows).reset_index(name='anomalous_count')

    # 결과를 병합하여 반환
    result = pd.merge(total_counts, anomalous_counts, on=['연', '지점명'], how='left')
    return result

actual_agg = {}

for i in ['peach365','pear365']:

  df= actual_result.copy().drop_duplicates()
  smx = count_anomalous_rows2(df,anomality = 'sarimax_anomality',prediction=i)
  smx.rename(columns = {'anomalous_count':'sarimax'},inplace =True)

  iso = count_anomalous_rows2(df,anomality = 'isoforest_anomality',prediction=i)
  iso.rename(columns = {'anomalous_count':'isoforest'},inplace =True)

  actual_agg[i]= pd.merge(smx,iso,on = ['지점명','total_count','연'],how='left')

In [58]:
# 그 값을 기준으로 정렬
sorted_peach = actual_agg['peach365'].groupby('지점명')[['sarimax', 'isoforest']].mean().reset_index()
sorted_peach = sorted_peach.sort_values(by=['sarimax', 'isoforest'],ascending=False) ; sorted_peach

,지점명,sarimax,isoforest
39,성산,6.640000,10.080000
16,대구(기),5.000000,7.500000
65,제주,4.678571,5.964286
80,포항,4.060606,5.606061
28,부산,4.037037,5.555556
...,...,...,...
24,백령도,0.857143,1.238095
27,봉화,0.800000,0.700000
79,파주,0.692308,1.230769
77,태백,0.450000,0.400000


In [59]:
sorted_pear = actual_agg['pear365'].groupby('지점명')[['sarimax', 'isoforest']].mean().reset_index()
sorted_pear = sorted_pear.sort_values(by=['sarimax', 'isoforest'],ascending=False) ; sorted_pear.head(50)

,지점명,sarimax,isoforest
16,대구(기),4.500000,6.500000
65,제주,3.964286,4.892857
39,성산,3.920000,6.280000
23,밀양,3.400000,4.250000
28,부산,3.370370,4.740741
80,포항,3.272727,4.393939
2,거제,3.150000,4.400000
15,대구,3.142857,3.885714
46,여수,3.030303,3.848485
51,완도,3.000000,4.346154


In [64]:
condition = (actual_result['지점명']=='성산')
condition2 = (actual_result['isoforest_anomality']==-1)
condition3 = (actual_result['날짜']>actual_result['peach365'])


actual_result[condition & condition2 & condition3].drop_duplicates()

,일 최심적설(cm),일시,합계 3시간 신적설(cm),안개 계속시간(hr),일강수량(mm),일 최심신적설(cm),지점,지점명,평균기온(°C),최저기온(°C),...,평균 전운량(1/10),평균 지면온도(°C),최저 초상온도(°C),해양,연,sarimax_anomality,isoforest_anomality,peach365,pear365,날짜
140487,0.0,1989-04-05,0.0,0.0,0.0,0.0,187.0,성산,14.0,11.6,...,2.8,15.5,5.5,0,1989,-1,-1,89,84.0,95
140495,0.0,1989-04-13,0.0,0.0,0.0,0.0,187.0,성산,13.9,6.6,...,6.5,17.0,2.7,0,1989,1,-1,89,84.0,103
140496,0.0,1989-04-14,0.0,0.0,32.8,0.0,187.0,성산,16.0,14.9,...,10.0,15.3,12.9,0,1989,-1,-1,89,84.0,104
140498,0.0,1989-04-16,0.0,0.0,0.0,0.0,187.0,성산,14.9,10.5,...,2.5,14.5,6.3,0,1989,1,-1,89,84.0,106
140499,0.0,1989-04-17,0.0,0.0,0.0,0.0,187.0,성산,16.5,5.1,...,0.3,16.7,0.3,0,1989,-1,-1,89,84.0,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145600,0.0,2012-04-20,0.0,0.0,12.0,0.0,188.0,성산,15.7,14.8,...,10.0,15.5,14.3,0,2012,1,-1,88,90.0,111
145663,0.0,2013-03-25,0.0,0.0,0.0,0.0,188.0,성산,5.6,3.6,...,3.4,7.6,3.1,0,2013,-1,-1,75,88.0,84
145676,0.0,2013-04-07,0.0,0.0,0.0,0.0,188.0,성산,7.0,5.6,...,4.0,7.5,5.0,0,2013,-1,-1,75,88.0,97
145689,0.0,2013-04-20,0.0,0.0,4.1,0.0,188.0,성산,10.0,7.7,...,9.4,10.3,7.4,0,2013,1,-1,75,88.0,110
